In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="../data/external",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="../data/external",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████████████████████| 26421880/26421880 [00:25<00:00, 1034644.05it/s]


Extracting ../data/external/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/external/FashionMNIST/raw



100%|█████████████████████████████████| 29515/29515 [00:00<00:00, 158804.02it/s]


Extracting ../data/external/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/external/FashionMNIST/raw



100%|█████████████████████████████| 4422102/4422102 [00:12<00:00, 346696.07it/s]


Extracting ../data/external/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/external/FashionMNIST/raw



100%|█████████████████████████████████| 5148/5148 [00:00<00:00, 13444755.29it/s]

Extracting ../data/external/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/external/FashionMNIST/raw



In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [18]:
from tqdm import tqdm
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in tqdm(enumerate(dataloader), total= len(dataloader), desc="Processing images"):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #if batch % 100 == 0:
        #loss, current = loss.item(), (batch + 1) * len(X)
        #print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


Processing images: 100%|█████████████████████| 938/938 [00:04<00:00, 191.32it/s]


Test Error: 
 Accuracy: 76.6%, Avg loss: 0.672500 

Epoch 2
-------------------------------


Processing images: 100%|█████████████████████| 938/938 [00:04<00:00, 197.69it/s]


Test Error: 
 Accuracy: 77.3%, Avg loss: 0.658649 

Epoch 3
-------------------------------


...

Test Error: 
 Accuracy: 77.7%, Avg loss: 0.645826 

Epoch 4
-------------------------------


Processing images: 100%|█████████████████████| 938/938 [00:04<00:00, 196.33it/s]


Test Error: 
 Accuracy: 78.2%, Avg loss: 0.633948 

Epoch 5
-------------------------------


Processing images: 100%|█████████████████████| 938/938 [00:04<00:00, 194.52it/s]


Test Error: 
 Accuracy: 78.5%, Avg loss: 0.622963 

Done!
